In [1]:
import os 
import torch
import torchvision
import torch.nn
from torch.autograd import Variable
from torch.autograd.gradcheck import zero_gradients
from torch.autograd import Variable
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
import math
import sys
sys.path.append('../../')
from utils import *
import sys
sys.path.append('../')
from output_utils import *

os.environ['CUDA_DEVICE_ORDER']="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES']='5' 

In [2]:
def YALEBXF_loader(batch_size=20, test_batch_size=2):
    train_data, train_label, test_data, test_label = YALE_split('../../datasets/yale/YALEBXF.mat') 
    train_data = train_data.transpose((0, 3, 1, 2) )
    test_data = test_data.transpose((0, 3, 1, 2) )
    tensor_x = torch.stack([torch.FloatTensor(i) for i in train_data]) # transform to torch tensors
    tensor_y = torch.stack([torch.LongTensor([i]) for i in train_label])[:,0]
    train_dataset = torch.utils.data.TensorDataset(tensor_x,tensor_y)
    
    tensor_x = torch.stack([torch.FloatTensor(i) for i in test_data]) # transform to torch tensors
    tensor_y = torch.stack([torch.LongTensor([i]) for i in test_label])[:,0]
    test_dataset = torch.utils.data.TensorDataset(tensor_x,tensor_y)
    
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
        shuffle=True, pin_memory=True)
    
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size,
        shuffle=False, pin_memory=True)
    return train_loader, test_loader

In [3]:
train_loader, test_loader = YALEBXF_loader(100, 100)
num_step = 20
max_perturb = 0.0314
step_size = max_perturb*2.5/num_step

model = torch.load('./model_robust.net')
model.cuda();
model.eval();
output_train_benign, output_train_adversarial, output_test_benign, output_test_adversarial, train_label, test_label =\
classifier_performance(model, train_loader, test_loader, num_step, max_perturb, step_size, clip_min=0, clip_max=1.0)

results_dir = './output_results'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
np.savez(os.path.join(results_dir, 'model_robust.npz'), 
         output_train_benign = output_train_benign, output_test_benign = output_test_benign,
         output_train_adversarial = output_train_adversarial, output_test_adversarial = output_test_adversarial,
         train_label = train_label, test_label = test_label)

using cuda
set up cuda
Benign accuracy:  (0.99263157894736842, 0.83268482490272377)  Adversarial accuracy:  (0.85684210526315785, 0.50389105058365757)
